In [1]:
import os;
#collect all the filenames and ready to read in
def filenames():
    files =[]
    for file in os.listdir('../Sports'):
        if ".json.gz" in file:
            files.append(file)
    return files

In [2]:
import gzip
#read single sport
def readfiles(sport):
    sportdata =[]
    print("start %s"%sport)
    with gzip.open(sport, 'r') as f:
        for l in f:
            l = l.decode('ascii')
            user = eval(l)
            sportdata.append(user) 
                
        print("finish %s"%sport)
    return sportdata;

In [4]:
sports = filenames()
sports_dictionary = {}
for sport in sports:
    sports_dictionary[sport]= readfiles('../Sports/' + sport)

start ../Sports/aerobics.json.gz
finish ../Sports/aerobics.json.gz
start ../Sports/badminton.json.gz
finish ../Sports/badminton.json.gz
start ../Sports/basketball.json.gz
finish ../Sports/basketball.json.gz
start ../Sports/beach volleyball.json.gz
finish ../Sports/beach volleyball.json.gz
start ../Sports/bike (transport).json.gz
finish ../Sports/bike (transport).json.gz
start ../Sports/bike.json.gz
finish ../Sports/bike.json.gz
start ../Sports/circuit training.json.gz
finish ../Sports/circuit training.json.gz
start ../Sports/climbing.json.gz
finish ../Sports/climbing.json.gz
start ../Sports/core stability training.json.gz
finish ../Sports/core stability training.json.gz
start ../Sports/cross-country skiing.json.gz
finish ../Sports/cross-country skiing.json.gz
start ../Sports/dancing.json.gz
finish ../Sports/dancing.json.gz
start ../Sports/downhill skiing.json.gz
finish ../Sports/downhill skiing.json.gz
start ../Sports/elliptical.json.gz
finish ../Sports/elliptical.json.gz
start ../Spor

In [5]:
#this function takes in one single workout and forms a list of triple in the 
#form [[1,2,3], [1,2,3]] where 1, 2, 3 are the variables like longitude, latitude, and altitude
#these variables can be chosen randomly
#return a list of a triple of variables
def buildtriple(workout, variable1, variable2, variable3):
    triple_list =[]
    #print("%s has %d"%(variable1,len(workout[variable1])))
    #print("%s has %d"%(variable2,len(workout[variable2])))
    #print("%s has %d"%(variable3,len(workout[variable3])))
    for i in range(len(workout[variable1])):    
        sub =[workout[variable1][i],workout[variable2][i],workout[variable3][i]]
        triple_list.append(sub)
    return triple_list

In [6]:
from haversine import haversine
#return {abc:[[1,2,3],[1,2,3],....., [1,2,3]], dce:[[1,2,3],[1,2,3],....., [1,2,3]], ......}
#sport is list
def tripleforAllworkouts(sport, variable1, variable2, variable3):
    workoutListForOneSport = {}
    for workout in sport:
        #temp is a dictionary for one workout
        temp= buildtriple(workout, variable1, variable2, variable3)
        #workoutListForOneSport is a list of dictionaries of workouts
        workoutListForOneSport[workout['id']] = temp
    return workoutListForOneSport

#workout is a list[[,,,],[,,,],[]....]
def calculateDistance(workout):
    distancelist = []
    start=[workout[0][0], workout[0][1]]
    
    for index in range(1,len(workout)):
        current = [workout[index][0], workout[index][1]]
        distance = haversine(start,current,miles = True) 
        distancelist.append(distance)
        start= current
    return distancelist

def calculate_time_interval (workout):
    time_interval_list = []
    timestamp_list = workout["timestamp"]
    for i in range(0,len(timestamp_list)-1):
        time_interval = timestamp_list[i+1] - timestamp_list[i] # in seconds
        time_interval_list.append(time_interval)
    return time_interval_list

In [7]:
#return a pair of duistance and time
def pair_disandtime(sport):
    #construct the dictioanry of the time intevral for one sport
    time_interval ={}
    for workout in sport:
        time_interval[workout['id']] = calculate_time_interval(workout)
    print("time interval has been stored")
    
    # form a distance list for one sport
    #triplelist contains the location info
    print(len(sport))
    #triplelist is a list of the dictoanaries of workouts
    triplelist = tripleforAllworkouts(sport, "latitude", "longitude", "altitude")
    #distanceList is a list that has lists of distance for each workout for one sport
    distanceList = {}    
    #a workout is the jkey
    for workout in triplelist:
        list_dis = calculateDistance(triplelist[workout])
        distanceList[workout] = list_dis
    print("distance has been stored")
    pair = [distanceList, time_interval]
    return pair

In [8]:
def has0timeinterval(list_time):
    for time in list_time:
        if time is 0:
            return 1
    return 0

In [9]:
#return a list of speed for all the workouts for a sport
def get_speedlist(pair):
    time_0 = []
    time = pair[1] # dict
    distance = pair[0] #dict
    speed_list = []
    for ID in time:
        single_workout = []
        list_time = time[ID]
        if has0timeinterval(list_time) is 0:
            for k in range(len(distance[ID])):
                single_workout.append(distance[ID][k]*3600/list_time[k])
            speed_list.append(single_workout)
        else:
            time_0.append(ID)
                
    result =[speed_list, time_0]
    return result

In [10]:
#write all the 
cleanedout = {}
with gzip.open("calculated_speed.json.gz", 'a') as file:
    for sport in sports_dictionary:
        print("start wriitng %s" % sport)
        file.write(bytes(sport+'\n', 'ascii'))
        pair = pair_disandtime(sports_dictionary[sport])
        data = get_speedlist(pair)
        speedlist = data[0]
        cleanedout[sport] = data[1]
        for workout in speedlist:
            file.write(bytes(str(workout)+'\n','ascii'))
    
    print("finsihed wriitng %s" % sport)

start wriitng soccer.json.gz
time interval has been stored
71
distance has been stored
start wriitng skate.json.gz
time interval has been stored
512
distance has been stored
start wriitng bike (transport).json.gz
time interval has been stored
10445
distance has been stored
start wriitng hiking.json.gz
time interval has been stored
357
distance has been stored
start wriitng golf.json.gz
time interval has been stored
12
distance has been stored
start wriitng swimming.json.gz
time interval has been stored
41
distance has been stored
start wriitng circuit training.json.gz
time interval has been stored
218
distance has been stored
start wriitng weight training.json.gz
time interval has been stored
237
distance has been stored
start wriitng basketball.json.gz
time interval has been stored
14
distance has been stored
start wriitng core stability training.json.gz
time interval has been stored
1262
distance has been stored
start wriitng orienteering.json.gz
time interval has been stored
1197
di

KeyboardInterrupt: 